In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import shuffle
from music21 import *
from glob import glob
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, Dropout
from keras.utils import np_utils

%run lstm_model.ipynb
np.set_printoptions(threshold=1000000)
bach_chorale_path = 'datasets/bach2'
parts_index = [0,1,2,3]
#parts_index = [0,1]
note_type = set()
"""
note_type:
0-127 : note pitch
128: slur-1st-part
129: slur-2nd-part
130: slur-3rd-part
131: slur-4th-part
132: rest
133: start symbol
134: end symbol
"""

def filter_files(files):
    file_list = []
    file_count = 0
    for i, file in enumerate(files, 1):
        s = converter.parse(file)
        if len(s.parts) == len(parts_index):
            print(i, file)
            file_count += 1
            file_list.append(file)
    return file_list, file_count

def chorale_to_inputs(s,i):
    inputs = []
    for n in s.parts[i].flat.notes:
#         if n.isNote:
#             print(n.nameWithOctave, int(4 * n.quarterLength), n.offset)
#         else:
#             print(n.pitchNames, int(4 * n.quarterLength), n.offset)
        #print(n.expressions)
        if n.isNote:
            inputs.append([n.pitch.midi])
            note_type.add(frozenset([n.pitch.midi]))
        elif n.isChord:
            inputs.append([_note.pitch.midi for _note in n])
            note_type.add(frozenset([_note.pitch.midi for _note in n]))
        l = n.quarterLength
#         if l == 0:
#             l = 1
        for j in range(1, int(4 * l)):
            if i == 0:
                inputs.append([128])
                note_type.add(frozenset([128]))
            elif i == 1:
                inputs.append([129])
                note_type.add(frozenset([129]))
            elif i == 2:
                inputs.append([130])
                note_type.add(frozenset([130]))
            elif i == 3:
                inputs.append([131])
                note_type.add(frozenset([131]))
    return inputs

def make_time_metadata(file):
    chorale = converter.parse(file)
    chorale_length = int(chorale.parts[0].duration.quarterLength * 4)
    time_metadata = np.zeros((chorale_length, ))
    
    for i in range(0, chorale_length):
        time_metadata[i] = i + 1
    return np.array(time_metadata, dtype=np.int32)

def append_notes(max_length, part_i_length, part_i_input):
    note_type.add(frozenset([132]))# 132-->note.Rest
    for num_appended_notes in range(max_length - part_i_length):
        part_i_input.append([132])
    return part_i_input

def check_parts_length(parts_length, inputs):
    max_length = max(parts_length)
    for i in parts_index:
        if parts_length[i] < max_length:
            inputs[i] = append_notes(max_length, parts_length[i], inputs[i])
                
def make_dataset(file_list):
    X = []
    time_metadata = []
    for file in tqdm(file_list):
        s = converter.parse(file)
        
        inputs = []
        parts_length = []
        time_metadata.append(make_time_metadata(file))
        for i in parts_index:
            part_input = chorale_to_inputs(s,i)
            parts_length.append(len(part_input))
            inputs.append(part_input)
        check_parts_length(parts_length, inputs)
        np_inputs = np.array(inputs)
        X.append(np_inputs)
    #X.reshape(X.shape[1:])
    #print(time_metadata)
    return X
def generator_X(notes):
    t = 0
    while True:
        X = []
        if t < notes.shape[0] - 16:
            X.append(notes[t:t+16])
        else: 
            t = 0
            continue
        t += 16
        X = np.array(X)
        yield (X)
def generate_notes(lstm_model, X):
    output_indices = []
    for i in range(len(parts_index)):
        model = load_model("models/bach"+str(i)+".h5")
#     model = load_model(lstm_model.name+".h5")
        gen = ((X) for (X) in generator_X(lstm_model.X))
#     predict_list = model.predict(X, batch_size=32)
        predict_list = model.predict_generator(gen, steps=300)
        part_notes = []
        for time in range(predict_list.shape[0]):
            for note_index in range(predict_list.shape[1]):
                part_notes.append(np.argmax(predict_list[time][note_index]))
        output_indices.append(part_notes)
    output_indices = np.array(output_indices)
    output_chorale = index_note_transform(output_indices, "to_note", len(parts_index))
    #output_indices = np.transpose(output_indices)
   # output_chorale = np.transpose(output_chorale)
    midi = chorale_to_midi(output_chorale)
    print(output_chorale)
    return midi

# def insert_note_chord(n, d, new_note, part):
    
# def notes_to_midi(n, d, new_note, part):
#     if (len(n) < 2): # note or rest
#         if n[0] == 132:
#             part.append(note.Rest())
#         elif n[0] < 128:
#             if d > 0:
#                 new_note.duration = duration.Duration(d / 4)
#                 part.append(new_note)
#                 d = 1
#                 new_note = note.Note(n[0])
#             else:
#                 d += 1
#     else: # chord
#         if d > 0:
#             new_note.duration = duration.Duration(d / 4)
#             part.append(new_note)
#             d = 1
#             new_note = chord.Chord(n)
#         else:
#             d += 1
def chorale_to_midi(chorale):
    score = stream.Score()
    for i, chorale_part in enumerate(chorale):
        part = stream.Part(id=i)
        d = 0
        new_note = note.Rest()
        for sound in chorale_part:
            sound = [n for n in sound]
            if (len(sound) < 2): # note or rest
                if sound[0] == 132:
                    part.append(note.Rest())
                elif sound[0] < 128:
                    if d > 0:
                        new_note.duration = duration.Duration(d / 4)
                        part.append(new_note)
                    d = 1
                    new_note = note.Note(sound[0])
                else:
                    d += 1
            else: # chord
                if d > 0:
                    new_note.duration = duration.Duration(d / 4)
                    part.append(new_note)
                d = 1
                new_note = chord.Chord(sound)
            
        new_note.duration = duration.Duration(d / 4)
        part.append(new_note)
        score.insert(part)
    return score
def make_note_type_list():
    note_type_list = []
    for i in note_type:
        note_type_list.append(i)
    return sorted(note_type_list)
# def transform_note_to_index(notes):
#     for time in range(notes.shape[0]):
#         for p_index in range(4):
#             notes[time][p_index] = note_dict[notes[time][p_index]]
#     return notes
def index_note_transform(inputs, option, pn):
    transformed_inputs = []
    if option == "to_note":
        note_dict = dict((index, note) for index, note in enumerate(note_type))
    elif option == "to_index":
        note_dict = dict((note, index) for index, note in enumerate(note_type))
    
    for p_index in range(pn):
        transformed_part = []
        for n in range(inputs.shape[1]):
            for keys in note_dict:
                if option == "to_note":
                    if inputs[p_index][n] == keys:
                        transformed_part.append(note_dict[keys])
                elif option == "to_index":
                    if frozenset(inputs[p_index][n]) == keys:
                        transformed_part.append(note_dict[keys])
        transformed_inputs.append(transformed_part)
    return np.array(transformed_inputs)
def notes_to_piano_roll(dataset_notes):
    start_symbol = np.full((16, 1),133)
    end_symbol = np.full((16, 1),134)
    note_type.add(frozenset([133]))
    note_type.add(frozenset([134]))
    for chorale_index in range(len(dataset_notes)):
        piano_roll_notes = []
        chorale = np.array(dataset_notes[chorale_index])
        for i in range(len(parts_index)):
            piano_roll_notes.append(np.concatenate((start_symbol, chorale[i], end_symbol)))
        piano_roll_notes = np.array(piano_roll_notes)
        piano_roll_notes = index_note_transform(piano_roll_notes, "to_index", len(parts_index))
        piano_roll_notes = np.transpose(piano_roll_notes)
        #piano_roll_notes = index_note_transform(transposed_chorale, "to_index", 4)
        yield piano_roll_notes
def notes_to_onehot(notes):
    note_dict = note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    print(note_dict)
    for i, n in enumerate(notes):
        a = []
        for note in n:
            a.append(np.array(note == np.arange(0, len(note_dict)), dtype=np.int32))
        onehot.append(a)
    return np.array(onehot)
def part_notes_to_onehot(notes):
    note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = np.zeros(len(note_dict,),dtype=np.int32)
        for note in n:
            a[note] = 1
        onehot.append(a)
    return np.array(onehot)

def load_files():
    if os.path.exists(bach_chorale_path):
#         file_list = filter_files(glob(bach_chorale_path))
        file_list, num_file = filter_files(glob(bach_chorale_path + '/*.mxl') +
                                 glob(bach_chorale_path + '/*.mid'))
        print(num_file)
        dataset_notes = make_dataset(file_list)
        shuffle(dataset_notes)
        chorale_list = [notes for chorale_index, notes in enumerate(notes_to_piano_roll(dataset_notes))]
        notes = chorale_list[0]
        for i, n in enumerate(chorale_list):
            if i != 0:
                notes = np.concatenate((notes, n))
        print(notes.shape)
        Y_onehot_notes = notes_to_onehot(notes)
        X_onehot_notes = part_notes_to_onehot(notes)
#         notes = notes[:,:,np.newaxis]
        for i in range(len(parts_index)):
            lstm_model = LSTM_Model(X=X_onehot_notes,Y=Y_onehot_notes,epochs=5, output_dim=len(note_type), name="bach"+str(i))
            gen = ((X,Y) for (X, Y) in lstm_model.generator(X_onehot_notes, Y_onehot_notes, i))
            lstm_model.model2()
            lstm_model.train_model(gen)

        score = generate_notes(lstm_model, notes)
                
        output_file = str(lstm_model.epochs) + '-epochs_' + str(num_file) + '-samples.mid'
        mf = midi.translate.music21ObjectToMidiFile(score)
        mf.open(output_file, 'wb')
        mf.write()
        mf.close()
        print("File " + output_file + " written")
    else:
        print("The file path is wrong!!")
    
def main():
    load_files()
if __name__ == "__main__":
    main()

Using TensorFlow backend.


1 datasets/bach2\bwv292.mxl
2 datasets/bach2\bwv293.mxl
3 datasets/bach2\bwv294.mxl
4 datasets/bach2\bwv295.mxl
5 datasets/bach2\bwv296.mxl
6 datasets/bach2\bwv297.mxl
7 datasets/bach2\bwv298.mxl
8 datasets/bach2\bwv299.mxl
9 datasets/bach2\bwv300.mxl
10 datasets/bach2\bwv301.mxl
11 datasets/bach2\bwv302.mxl
12 datasets/bach2\bwv303.mxl
13 datasets/bach2\bwv304.mxl
14 datasets/bach2\bwv305.mxl
15 datasets/bach2\bwv306.mxl
16 datasets/bach2\bwv307.mxl
17 datasets/bach2\bwv308.mxl
18 datasets/bach2\bwv309.mxl
19 datasets/bach2\bwv310.mxl
20 datasets/bach2\bwv311.mxl
21 datasets/bach2\bwv312.mxl
22 datasets/bach2\bwv313.mxl
23 datasets/bach2\bwv314.mxl
24 datasets/bach2\bwv315.mxl
25 datasets/bach2\bwv316.mxl
26 datasets/bach2\bwv317.mxl
27 datasets/bach2\bwv318.mxl
28 datasets/bach2\bwv319.mxl
29 datasets/bach2\bwv320.mxl
30 datasets/bach2\bwv321.mxl
31 datasets/bach2\bwv322.mxl
32 datasets/bach2\bwv323.mxl
33 datasets/bach2\bwv324.mxl
34 datasets/bach2\bwv325.mxl
35 datasets/bach2\bwv32

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:05<00:00,  5.75it/s]


(12640, 4)
{0: frozenset({49}), 1: frozenset({38}), 2: frozenset({79}), 3: frozenset({77}), 4: frozenset({50}), 5: frozenset({75}), 6: frozenset({72}), 7: frozenset({78}), 8: frozenset({68}), 9: frozenset({63}), 10: frozenset({44}), 11: frozenset({59}), 12: frozenset({56}), 13: frozenset({133}), 14: frozenset({131}), 15: frozenset({128}), 16: frozenset({81}), 17: frozenset({70}), 18: frozenset({57}), 19: frozenset({53}), 20: frozenset({58}), 21: frozenset({51}), 22: frozenset({48}), 23: frozenset({65}), 24: frozenset({54}), 25: frozenset({76}), 26: frozenset({130}), 27: frozenset({41}), 28: frozenset({39}), 29: frozenset({42}), 30: frozenset({60}), 31: frozenset({132}), 32: frozenset({61}), 33: frozenset({73}), 34: frozenset({62}), 35: frozenset({71}), 36: frozenset({52}), 37: frozenset({69}), 38: frozenset({74}), 39: frozenset({67}), 40: frozenset({64}), 41: frozenset({134}), 42: frozenset({47}), 43: frozenset({45}), 44: frozenset({43}), 45: frozenset({40}), 46: frozenset({46}), 47: f

256/256 [==============================] - ETA: 9s - loss: 0.460 - ETA: 7s - loss: 0.442 - ETA: 8s - loss: 0.459 - ETA: 8s - loss: 0.451 - ETA: 8s - loss: 0.430 - ETA: 8s - loss: 0.487 - ETA: 8s - loss: 0.476 - ETA: 8s - loss: 0.464 - ETA: 8s - loss: 0.452 - ETA: 8s - loss: 0.441 - ETA: 8s - loss: 0.436 - ETA: 8s - loss: 0.431 - ETA: 8s - loss: 0.417 - ETA: 7s - loss: 0.412 - ETA: 7s - loss: 0.402 - ETA: 7s - loss: 0.397 - ETA: 7s - loss: 0.386 - ETA: 7s - loss: 0.380 - ETA: 7s - loss: 0.375 - ETA: 7s - loss: 0.366 - ETA: 7s - loss: 0.358 - ETA: 7s - loss: 0.364 - ETA: 7s - loss: 0.356 - ETA: 6s - loss: 0.359 - ETA: 6s - loss: 0.351 - ETA: 6s - loss: 0.351 - ETA: 6s - loss: 0.346 - ETA: 6s - loss: 0.344 - ETA: 6s - loss: 0.337 - ETA: 6s - loss: 0.332 - ETA: 6s - loss: 0.328 - ETA: 6s - loss: 0.347 - ETA: 6s - loss: 0.357 - ETA: 6s - loss: 0.375 - ETA: 6s - loss: 0.386 - ETA: 5s - loss: 0.394 - ETA: 5s - loss: 0.405 - ETA: 5s - loss: 0.411 - ETA: 5s - loss: 0.414 - ETA: 5s - loss: 0.413

...

256/256 [==============================] - ETA: 6s - loss: 0.307 - ETA: 7s - loss: 0.156 - ETA: 7s - loss: 0.314 - ETA: 7s - loss: 0.287 - ETA: 7s - loss: 0.285 - ETA: 7s - loss: 0.240 - ETA: 7s - loss: 0.220 - ETA: 7s - loss: 0.199 - ETA: 7s - loss: 0.226 - ETA: 7s - loss: 0.207 - ETA: 7s - loss: 0.198 - ETA: 7s - loss: 0.196 - ETA: 7s - loss: 0.209 - ETA: 7s - loss: 0.219 - ETA: 7s - loss: 0.221 - ETA: 7s - loss: 0.224 - ETA: 7s - loss: 0.214 - ETA: 6s - loss: 0.212 - ETA: 6s - loss: 0.213 - ETA: 6s - loss: 0.232 - ETA: 6s - loss: 0.250 - ETA: 6s - loss: 0.267 - ETA: 6s - loss: 0.275 - ETA: 6s - loss: 0.286 - ETA: 6s - loss: 0.282 - ETA: 6s - loss: 0.278 - ETA: 6s - loss: 0.280 - ETA: 6s - loss: 0.275 - ETA: 6s - loss: 0.268 - ETA: 6s - loss: 0.269 - ETA: 6s - loss: 0.268 - ETA: 6s - loss: 0.265 - ETA: 6s - loss: 0.262 - ETA: 5s - loss: 0.263 - ETA: 5s - loss: 0.265 - ETA: 5s - loss: 0.267 - ETA: 5s - loss: 0.263 - ETA: 5s - loss: 0.258 - ETA: 5s - loss: 0.258 - ETA: 5s - loss: 0.253

256/256 [==============================] - ETA: 273s - loss: 3.911 - ETA: 96s - loss: 3.887 - ETA: 60s - loss: 3.81 - ETA: 44s - loss: 3.71 - ETA: 36s - loss: 3.57 - ETA: 30s - loss: 3.40 - ETA: 29s - loss: 3.31 - ETA: 25s - loss: 3.22 - ETA: 23s - loss: 3.18 - ETA: 21s - loss: 3.16 - ETA: 19s - loss: 3.07 - ETA: 18s - loss: 2.98 - ETA: 17s - loss: 2.90 - ETA: 16s - loss: 2.85 - ETA: 15s - loss: 2.76 - ETA: 14s - loss: 2.72 - ETA: 14s - loss: 2.66 - ETA: 13s - loss: 2.66 - ETA: 12s - loss: 2.64 - ETA: 12s - loss: 2.60 - ETA: 12s - loss: 2.55 - ETA: 11s - loss: 2.50 - ETA: 11s - loss: 2.46 - ETA: 11s - loss: 2.43 - ETA: 10s - loss: 2.39 - ETA: 10s - loss: 2.36 - ETA: 10s - loss: 2.35 - ETA: 10s - loss: 2.34 - ETA: 9s - loss: 2.3169 - ETA: 9s - loss: 2.275 - ETA: 9s - loss: 2.245 - ETA: 9s - loss: 2.217 - ETA: 8s - loss: 2.179 - ETA: 8s - loss: 2.152 - ETA: 8s - loss: 2.122 - ETA: 8s - loss: 2.086 - ETA: 8s - loss: 2.058 - ETA: 8s - loss: 2.045 - ETA: 8s - loss: 2.019 - ETA: 7s - loss: 1

256/256 [==============================] - ETA: 5s - loss: 0.155 - ETA: 6s - loss: 0.471 - ETA: 6s - loss: 0.450 - ETA: 6s - loss: 0.438 - ETA: 7s - loss: 0.420 - ETA: 7s - loss: 0.379 - ETA: 7s - loss: 0.370 - ETA: 7s - loss: 0.337 - ETA: 6s - loss: 0.321 - ETA: 6s - loss: 0.314 - ETA: 6s - loss: 0.288 - ETA: 6s - loss: 0.268 - ETA: 6s - loss: 0.262 - ETA: 6s - loss: 0.261 - ETA: 6s - loss: 0.259 - ETA: 6s - loss: 0.248 - ETA: 6s - loss: 0.240 - ETA: 6s - loss: 0.231 - ETA: 6s - loss: 0.222 - ETA: 6s - loss: 0.230 - ETA: 6s - loss: 0.227 - ETA: 6s - loss: 0.224 - ETA: 6s - loss: 0.218 - ETA: 6s - loss: 0.213 - ETA: 6s - loss: 0.207 - ETA: 6s - loss: 0.201 - ETA: 5s - loss: 0.196 - ETA: 5s - loss: 0.192 - ETA: 5s - loss: 0.201 - ETA: 5s - loss: 0.195 - ETA: 5s - loss: 0.198 - ETA: 5s - loss: 0.194 - ETA: 5s - loss: 0.190 - ETA: 5s - loss: 0.188 - ETA: 5s - loss: 0.183 - ETA: 5s - loss: 0.180 - ETA: 5s - loss: 0.177 - ETA: 5s - loss: 0.184 - ETA: 5s - loss: 0.185 - ETA: 5s - loss: 0.182

256/256 [==============================] - ETA: 6s - loss: 0.110 - ETA: 6s - loss: 0.158 - ETA: 6s - loss: 0.095 - ETA: 7s - loss: 0.071 - ETA: 7s - loss: 0.124 - ETA: 7s - loss: 0.113 - ETA: 7s - loss: 0.097 - ETA: 7s - loss: 0.088 - ETA: 7s - loss: 0.081 - ETA: 7s - loss: 0.076 - ETA: 7s - loss: 0.071 - ETA: 7s - loss: 0.067 - ETA: 7s - loss: 0.063 - ETA: 7s - loss: 0.059 - ETA: 7s - loss: 0.055 - ETA: 6s - loss: 0.052 - ETA: 6s - loss: 0.049 - ETA: 6s - loss: 0.050 - ETA: 6s - loss: 0.048 - ETA: 6s - loss: 0.048 - ETA: 6s - loss: 0.046 - ETA: 6s - loss: 0.046 - ETA: 6s - loss: 0.044 - ETA: 6s - loss: 0.044 - ETA: 6s - loss: 0.042 - ETA: 6s - loss: 0.041 - ETA: 6s - loss: 0.040 - ETA: 6s - loss: 0.039 - ETA: 6s - loss: 0.038 - ETA: 6s - loss: 0.037 - ETA: 6s - loss: 0.036 - ETA: 6s - loss: 0.035 - ETA: 6s - loss: 0.034 - ETA: 5s - loss: 0.033 - ETA: 5s - loss: 0.032 - ETA: 5s - loss: 0.031 - ETA: 5s - loss: 0.030 - ETA: 5s - loss: 0.030 - ETA: 5s - loss: 0.029 - ETA: 5s - loss: 0.029

  frozenset({50}) frozenset({131}) frozenset({131}) frozenset({131})]]
File 5-epochs_40-samples.mid written


In [ ]:
n = [0,1,2,3,4,5,6,7,8]
n = np.array(n)
g = np.zeros((len(n),),dtype=np.int32)
print(g)
a = [0,5,7]
for i in a:
    print(np.array(i == np.arange(0, len(n) )))

In [86]:
def inner_pro(x1,x2,y1,y2):
    return (x1*y1)+(x2*y2)
x = [1.4, 1.6]
A1 = [1.5, 2.0, 1.6, 1.2, 1.5]
A2 = [1.7, 1.9, 1.8, 1.5, 1.0]
ans = []
for i in range(5):
    ans.append(inner_pro(x[0],x[1],A1[i],A2[i]) / (inner_pro(x[0],x[1],x[0],x[1]) * inner_pro(A1[i],A2[i],A1[i],A2[i])))
ans

[0.2074653076684687,
 0.16978126126545184,
 0.19530057979859625,
 0.2446219152457011,
 0.2518720217835262]

In [68]:
f = frozenset([1,2,3,4,5])
for i in f:
    print(i)

5
